In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [2]:
AGENT_NAME = "ab040-other-sdks"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.

from typing import cast
import uuid

from agentex.types import Task

TASK_ID = str(uuid.uuid4())[:8]

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="task/create",
    params={
        "name": f"{TASK_ID}-task",
        "params": {}
    }
)

task = cast(Task, rpc_response.result)
print(task)

Task(id='75c695e6-ba24-4a45-9843-2f6c70a16d79', created_at=datetime.datetime(2025, 7, 27, 1, 25, 7, 352235, tzinfo=TzInfo(UTC)), name='c1b4a675-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 7, 27, 1, 25, 7, 352235, tzinfo=TzInfo(UTC)))


In [4]:
# Test non streaming response
from typing import cast
from agentex.types import Event

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="event/send",
    params={
        "content": {"type": "text", "author": "user", "content": "Hello tell me the latest news about AI and AI startups"},
        "task_id": task.id,
    }
)

event = cast(Event, rpc_response.result)
print(event)

Event(id='de2e5231-53c7-488c-9ecb-9f2605dea9f6', agent_id='cd4b9256-144b-4aef-949b-f9000995013b', sequence_id=217, task_id='75c695e6-ba24-4a45-9843-2f6c70a16d79', content=TextContent(author='user', content='Hello tell me the latest news about AI and AI startups', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 27, 1, 25, 7, 716885, tzinfo=TzInfo(UTC)))


In [5]:
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=20,
)

╭───────────────────────── USER [07/27/2025 01:25:07] ─────────────────────────╮
│ Hello tell me the latest news about AI and AI startups                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/27/2025 01:25:20] ─────────────────────────╮
│ 🔧 Tool Request: web_search                                                  │
│                                                                              │
│ Arguments:                                                                   │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "input": "latest news AI AI startups",                                    │
│    "model": "gpt-4o",                                                        │
│    "type": "web_search_preview",                                             │
│    "search_context_size": "high"                                             │
│  }                                                                           │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/27/2025 01:25:20] ─────────────────────────╮
│ ✅ Tool Response: web_search                                                 │
│                                                                              │
│ Response:                                                                    │
│                                                                              │
│                                                                              │
│  {                                                                           │
│    "type": "text",                                                           │
│    "text": "The artificial intelligence (AI) startup landscape has           │
│  experienced significant developments recently, marked by substantial fundi  │
│  rounds, strategic partnerships, and notable acquisitions.\n\n**Major Fundi  │
│  Rounds:**\n\n- **Perplexity AI**: This AI search startup secured $100       │
│  million in its latest funding round, elevating its valuation to $18 billio  │
│  The company's rapid growth underscores robust investor interest in the      │
│  competitive AI search market.                                               │
│  ([pymnts.com](https://www.pymnts.com/news/artificial-intelligence/2025/thi  │
│  week-in-ai-ai-startups-hit-fundraising-gold/?utm_source=openai))\n\n-       │
│  **Thinking Machines**: Founded by former OpenAI CTO Mira Murati, Thinking   │
│  Machines raised $2 billion, achieving a valuation of $10 billion. The       │
│  startup focuses on developing multimodal AI capable of processing various   │
│  forms of data.                                                              │
│  ([pymnts.com](https://www.pymnts.com/news/artificial-intelligence/2025/thi  │
│  week-in-ai-ai-startups-hit-fundraising-gold/?utm_source=openai))\n\n-       │
│  **Anthropic**: Reports indicate that Anthropic is in discussions with       │
│  investors for funding that could value the company at $100 billion,         │
│  following a previous valuation of $61.5 billion after a $3.5 billion        │
│  fundraise earlier this year.                                                │
│  ([pymnts.com](https://www.pymnts.com/news/artificial-intelligence/2025/thi  │
│  week-in-ai-ai-startups-hit-fundraising-gold/?utm_source=openai))\n\n**Stra  │
│  gic Investments and Partnerships:**\n\n- **Nvidia's Role**: Nvidia has      │
│  solidified its position as a key player in the AI sector, participating in  │
│  numerous funding rounds and supporting the growth of several AI startups.   │
│  Notably, Nvidia invested in Reka AI, contributing to its unicorn status, a  │
│  has been involved in funding rounds for companies like OpenAI, xAI, and     │
│  Scale AI.                                                                   │
│  ([elpais.com](https://elpais.com/economia/2025-07-26/nvidia-el-mayor-gigan  │
│  -bursatil-de-la-historia-que-crea-unicornios-con-la-magia-de-la-ia.html?ut  │
│  source=openai))\n\n- **Amazon and Anthropic**: Amazon increased its         │
│  investment in AI startup Anthropic by an additional $4 billion, bringing i  │
│  total investment to $8 billion. This partnership designates Amazon Web      │
│  Services (AWS) as Anthropic's primary training partner, highlighting        │
│  Amazon's commitment to advancing AI technologies.                           │
│  ([apnews.com](https://apnews.com/article/7a5764907e8cf0c23117be9c710e9f6a?  │
│  m_source=openai))\n\n**Market Trends:**\n\n- **Surge in AI Startup          │
│  Funding**: In the first half of 2025, U.S. startup funding surged by 75.6%  │
│  reaching $162.8 billion, driven largely by the AI boom. AI-related          │
│  investments constituted 64.1% of the total deal value during this period.   │
│  ([reuters.com](https://www.reuters.com/business/us-ai-startups-see-funding  │
│  urge-while-more-vc-funds-struggle-raise-data-shows-2025-07-15/?utm_source=  │
│  enai))\n\n- **Global Investme

╭──────────────────────── AGENT [07/27/2025 01:25:21] ─────────────────────────╮
│ Here are some of the latest updates in the AI and AI startups ecosystem:     │
│                                                                              │
│                             Major Funding Rounds                             │
│                                                                              │
│  • Perplexity AI: The AI search startup raised $100 million, boosting its    │
│    valuation to $18 billion, reflecting strong investor interest in AI       │
│    search technologies.                                                      │
│  • Thinking Machines: Founded by former OpenAI CTO Mira Murati, it secured   │
│    $2 billion, valuing the company at $10 billion. The startup is working on │
│    multimodal AI.                                                            │
│  • Anthropic: Currently in discussions for a funding round that could value  │
│    it at $100 billion, following a $3.5 billion raise earlier this year.     │
│                                                                              │
│                    Strategic Investments and Partnerships                    │
│                                                                              │
│  • Nvidia: A key AI sector player, Nvidia has participated in various        │
│    funding rounds, including those for OpenAI, xAI, and Scale AI, and        │
│    invested in Reka AI, leading to its unicorn status.                       │
│  • Amazon and Anthropic: Amazon has increased its investment in AI startup   │
│    Anthropic by $4 billion, totaling $8 billion. Amazon Web Services (AWS)   │
│    is Anthropic’s primary training partner.                                  │
│                                                                              │
│                                Market Trends                                 │
│                                                                              │
│  • U.S. Startup Funding: In the first half of 2025, AI-driven funding surged │
│    by 75.6% to $162.8 billion, with AI-related investments forming a         │
│    significant portion.                                                      │
│  • Global VC Investments: AI startups made up 53% of all global venture      │
│    capital investments in the first half of 2025.                            │
│                                                                              │
│                             Notable Acquisitions                             │
│                                                                              │
│  • OpenAI: Set to acquire Jony Ive's AI devices startup, io, for $6.4        │
│    billion in an all-equity deal, marking a move into AI hardware.           │
│                                                                              │
│ These updates reveal a vibrant and rapidly evolving AI startup landscape,    │
│ characterized by significant financial investments, strategic                │
│ collaborations, and innovative development.                                  │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 20 seconds - returning collected messages
